In [ ]:
! conda update --all --yes

In [2]:
!python -m pip install pyradiomics


  Using cached pyradiomics-3.0.1-cp38-cp38-linux_x86_64.whl


In [ ]:
!  conda install -c radiomics pyradiomics

In [3]:
import os
import pandas as pd
from skimage import io, transform
from skimage.measure import label, regionprops
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage.transform import resize
from scipy import ndimage
import glob
import nibabel as nib
from nilearn import image
pd.options.display.max_colwidth = 100
import shutil
import copy
import radiomics
import SimpleITK as sitk
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
from radiomics import firstorder, getTestCase, glcm, glrlm, glszm, imageoperations, shape,ngtdm, featureextractor

!pip show pyradiomics|grep Version

Version: 3.0.1


In [ ]:
firstorder

In [4]:
path = "/home/rfyahya/Downloads/training_with_val_split(1).csv"
#path = "/home/rfyahya/Desktop/Rawan/brain/clean/visual studios/dataframes/testing_set_t2w.csv"
test_set = pd.read_csv(path)

In [5]:
test_set

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,BraTS21ID,MGMT_value,path,mri_type
0,0,172,172,239,0,/home/rfyahya/Desktop/Rawan/brain/clean/preprocessed/train/00239/FLAIR/FL_to_SRI.nii.gz,FLAIR
1,1,173,173,239,0,/home/rfyahya/Desktop/Rawan/brain/clean/preprocessed/train/00239/T1w/T1_to_SRI.nii.gz,T1w
2,2,174,174,239,0,/home/rfyahya/Desktop/Rawan/brain/clean/preprocessed/train/00239/T1wCE/T1CE_to_SRI.nii.gz,T1wCE
3,3,175,175,239,0,/home/rfyahya/Desktop/Rawan/brain/clean/preprocessed/train/00239/T2w/T2_to_SRI.nii.gz,T2w
4,4,180,180,241,0,/home/rfyahya/Desktop/Rawan/brain/clean/preprocessed/train/00241/FLAIR/FL_to_SRI.nii.gz,FLAIR
...,...,...,...,...,...,...,...
1659,1663,1855,1855,1009,0,/home/rfyahya/Desktop/Rawan/brain/clean/preprocessed/train/01009/T1w/T1_to_SRI.nii.gz,T1w
1660,1664,1856,1856,1010,0,/home/rfyahya/Desktop/Rawan/brain/clean/preprocessed/train/01010/T1wCE/T1CE_to_SRI.nii.gz,T1wCE
1661,1665,1857,1857,1010,0,/home/rfyahya/Desktop/Rawan/brain/clean/preprocessed/train/01010/FLAIR/FL_to_SRI.nii.gz,FLAIR
1662,1666,1858,1858,1010,0,/home/rfyahya/Desktop/Rawan/brain/clean/preprocessed/train/01010/T1w/T1_to_SRI.nii.gz,T1w


In [6]:
# ID = test_set["path"].apply(lambda l: l.split('/')[2])
# test_set.insert(5,'ID' ,ID )
# test_set



ID = test_set["path"].apply(lambda l: l.split('/')[6])
print(ID)
test_set.insert(5,'ID' ,ID )

test_set

0       clean
1       clean
2       clean
3       clean
4       clean
        ...  
1659    clean
1660    clean
1661    clean
1662    clean
1663    clean
Name: path, Length: 1664, dtype: object


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,BraTS21ID,MGMT_value,ID,path,mri_type
0,0,172,172,239,0,clean,/home/rfyahya/Desktop/Rawan/brain/clean/preprocessed/train/00239/FLAIR/FL_to_SRI.nii.gz,FLAIR
1,1,173,173,239,0,clean,/home/rfyahya/Desktop/Rawan/brain/clean/preprocessed/train/00239/T1w/T1_to_SRI.nii.gz,T1w
2,2,174,174,239,0,clean,/home/rfyahya/Desktop/Rawan/brain/clean/preprocessed/train/00239/T1wCE/T1CE_to_SRI.nii.gz,T1wCE
3,3,175,175,239,0,clean,/home/rfyahya/Desktop/Rawan/brain/clean/preprocessed/train/00239/T2w/T2_to_SRI.nii.gz,T2w
4,4,180,180,241,0,clean,/home/rfyahya/Desktop/Rawan/brain/clean/preprocessed/train/00241/FLAIR/FL_to_SRI.nii.gz,FLAIR
...,...,...,...,...,...,...,...,...
1659,1663,1855,1855,1009,0,clean,/home/rfyahya/Desktop/Rawan/brain/clean/preprocessed/train/01009/T1w/T1_to_SRI.nii.gz,T1w
1660,1664,1856,1856,1010,0,clean,/home/rfyahya/Desktop/Rawan/brain/clean/preprocessed/train/01010/T1wCE/T1CE_to_SRI.nii.gz,T1wCE
1661,1665,1857,1857,1010,0,clean,/home/rfyahya/Desktop/Rawan/brain/clean/preprocessed/train/01010/FLAIR/FL_to_SRI.nii.gz,FLAIR
1662,1666,1858,1858,1010,0,clean,/home/rfyahya/Desktop/Rawan/brain/clean/preprocessed/train/01010/T1w/T1_to_SRI.nii.gz,T1w


In [7]:
test_set["BraTS21ID"].astype('int64', copy=False)
p = int(test_set["BraTS21ID"][0])
p

239

In [8]:
# def Skewness_Core_T1w (image,mask):

#     image =sitk.ReadImage(image)
#     mask =sitk.ReadImage(mask)
#     firstOrderFeatures = firstorder.RadiomicsFirstOrder(image,mask)
#     firstOrderFeatures.enableAllFeatures()  # On the feature class level, all features are disabled by default.
#     firstOrderFeatures.calculateFeatures()
#     for (key,val) in six.iteritems(firstOrderFeatures.featureValues):
#         print("\t%s: %s" % (key, val))

In [9]:
def Skewness_Core_T1w (image,mask):
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False
    image =sitk.ReadImage(image)
    mask =sitk.ReadImage(mask)
    firstOrderFeatures1 = firstorder.RadiomicsFirstOrder(image,mask, label=2&3,**settings)
    firstOrderFeatures1.enableAllFeatures()  
    firstOrderFeatures1.calculateFeatures()
    Skewness = firstOrderFeatures1.featureValues.get("Skewness")
    x = Skewness.astype(np.float)
    print(Skewness)
    return x

In [10]:
def Energy_Oedema_T1W (image,mask):
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False
    image =sitk.ReadImage(image)
    mask =sitk.ReadImage(mask)
    firstOrderFeatures = firstorder.RadiomicsFirstOrder(image,mask, label=1,**settings)
    firstOrderFeatures.enableAllFeatures()  
    firstOrderFeatures.calculateFeatures()
    Energy = firstOrderFeatures.featureValues.get("Energy")
    return Energy

In [11]:
def GLCM_Contrast_Necrosis_FLAIR (image,mask):
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False
    image =sitk.ReadImage(image)
    mask =sitk.ReadImage(mask)
    glcmFeatures = glcm.RadiomicsGLCM(image,mask, label=3,**settings)
    glcmFeatures.enableAllFeatures() 
    glcmFeatures.calculateFeatures()
    Contrast = glcmFeatures.featureValues.get("Contrast")
    return Contrast


In [12]:
def GLSZM_Grey_Level_Variance_Enhanced_T1c (image,mask):
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False
    image =sitk.ReadImage(image)
    mask =sitk.ReadImage(mask)
    glszmFeatures1 = glszm.RadiomicsGLSZM(image,mask, label=3,**settings)
    glszmFeatures1.enableAllFeatures()  
    glszmFeatures1.calculateFeatures()
    GrayLevelVariance = glszmFeatures1.featureValues.get("GrayLevelVariance")
    return GrayLevelVariance

In [13]:
def GLSZM_LowGrayLevelZoneEmphasis_Oedema_T2w (image,mask):
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False
    image =sitk.ReadImage(image)
    mask =sitk.ReadImage(mask)
    glszmFeatures = glszm.RadiomicsGLSZM(image,mask, label=1,**settings)
    glszmFeatures.enableAllFeatures() 
    glszmFeatures.calculateFeatures()
    LowGrayLevelZoneEmphasis = glszmFeatures.featureValues.get("LowGrayLevelZoneEmphasis")
    return LowGrayLevelZoneEmphasis

In [14]:
def original_ngtdm_Busyness_Core_T2w (image,mask):
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False
    image =sitk.ReadImage(image)
    mask =sitk.ReadImage(mask)
    ngtdmFeatures = ngtdm.RadiomicsNGTDM(image,mask, label=2&3,**settings)
    ngtdmFeatures.enableAllFeatures() 
    ngtdmFeatures.calculateFeatures()
    Busyness = ngtdmFeatures.featureValues.get("Busyness")
    return Busyness

In [15]:

T1c_filename = "T1wCE/T1CE_to_SRI.nii.gz"
T1_filename = "T1w/T1_to_SRI.nii.gz"
T2_filename = "T2w/T2_to_SRI.nii.gz"
fl_filename = "FLAIR/FL_to_SRI.nii.gz"
mask_suffix = "/home/rfyahya/Desktop/Rawan/brain/clean/segmented_real_ids/train/BRATS_"

rootdir_onlysurv = "/home/rfyahya/Desktop/Rawan/brain/clean/preprocessed/train/"

osinfo = test_set

subjlist = list(osinfo["ID"])


overallfeatdf = pd.DataFrame(data=[],
                             columns=['Skewness_Core_T1w',
                                                   'Energy_Oedema_T1W',
                                                   'GLCM_Contrast_Necrosis_FLAIR',
                                                   'GLSZM_Grey Level Variance_Enhanced_T1c',
                                                   'GLSZM_Low Grey Level Zone Emphasis_Oedema_T2w',
                                                   'original_ngtdm_Busyness_Core_T2w'])

overallfeatdf.index.name = "ID"

Skewness =[]
Energy =[]
Contrast=[]
Variance=[]
Emphasis=[]
Busyness =[]

for subj in tqdm(subjlist):
   
    currsubjdir1 = os.path.join(rootdir_onlysurv+subj+'/T1wCE/T1CE_to_SRI.nii.gz')
    currsubjdir2 = os.path.join(rootdir_onlysurv+subj+'/T1w/T1_to_SRI.nii.gz')
    currsubjdir3 = os.path.join(rootdir_onlysurv+subj+'/T2w/T2_to_SRI.nii.gz')
    currsubjdir4 = os.path.join(rootdir_onlysurv+subj+'/FLAIR/FL_to_SRI.nii.gz')
    p = int(subj)
    p2= str("{:04d}".format(p))
    maskpath = os.path.join(mask_suffix+p2+'.nii.gz')
    print(maskpath)
    print(currsubjdir2)
    featureVector = Skewness_Core_T1w (currsubjdir2,maskpath)

    try:
        featureVector = Skewness_Core_T1w (currsubjdir2,maskpath)
        featureVector2 = Energy_Oedema_T1W (currsubjdir2,maskpath)
        featureVector3 = GLCM_Contrast_Necrosis_FLAIR(currsubjdir4,maskpath)
        featureVector4 = GLSZM_Grey_Level_Variance_Enhanced_T1c(currsubjdir1,maskpath)
        featureVector5 = GLSZM_LowGrayLevelZoneEmphasis_Oedema_T2w(currsubjdir3,maskpath)
        featureVector6 = original_ngtdm_Busyness_Core_T2w(currsubjdir3,maskpath)
    except:
        pass
        
    Skewness.append(featureVector)
    Energy.append(featureVector2)
    Contrast.append(featureVector3)
    Variance.append(featureVector4)
    Emphasis.append(featureVector5)
    Busyness.append(featureVector6)
    
    currfeaturedf = pd.DataFrame(data=[[Skewness,Energy,Contrast,Variance,Emphasis,Busyness]] ,index=[subj], columns=['Skewness_Core_T1w',
                                                   'Energy_Oedema_T1W',
                                                   'GLCM_Contrast_Necrosis_FLAIR',
                                                   'GLSZM_Grey Level Variance_Enhanced_T1c',
                                                   'GLSZM_Low Grey Level Zone Emphasis_Oedema_T2w',
                                                   'original_ngtdm_Busyness_Core_T2w'])
    print(currfeaturedf)

    overallfeatdf = overallfeatdf.append(currfeaturedf, sort =False)
overallfeatdf.to_csv('/home/asrar/Documents/Train_RAD.csv')

  0%|                                                  | 0/1664 [00:00<?, ?it/s]


ValueError: invalid literal for int() with base 10: 'clean'

In [ ]:
for i in Busyness:
    print (float(i))

In [ ]:
for i in Skewness:
    print (float(i))